In [2]:
import cv2
import numpy as np
import PIL
from PIL import Image

In [3]:
# 1. 特征提取
def extract_features(image):
    feature_extractor = cv2.xfeatures2d.SURF_create()
    keypoints, descriptors = feature_extractor.detectAndCompute(image, None)
    return keypoints, descriptors

# 2. 特征匹配
def match_features_bf(descriptors1, descriptors2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)
    return good_matches

def match_features_flann(descriptors1, descriptors2):
    # FLANN 参数设置
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    # 创建 FLANN 匹配器
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    # 使用 KNN 进行匹配
    # matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    # 构建FLANN匹配器
    # 使用 KNN 进行匹配
    # 检查描述符是否为 None
    if descriptors1 is not None and descriptors2 is not None:
        # 构建FLANN匹配器
        flann = cv2.FlannBasedMatcher(index_params, search_params)

        # 使用 KNN 进行匹配
        matches = flann.knnMatch(descriptors1.astype(np.float32), descriptors2.astype(np.float32), k=2)
        
        # 选择好的匹配
        good_matches = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)
    else:
        print("No descriptors found.")

    # 选择好的匹配
    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)

    return good_matches

# 3. 变换估计
def estimate_transform(keypoints1, keypoints2, good_matches):
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    # 使用RANSAC算法估计变换矩阵
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return M

#4. 变换应用
def apply_transform(image, M):
    # 应用估计的变换矩阵到图像
    transformed_image = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))
    return transformed_image

In [4]:
# 一次配准
image1 = cv2.imread('origin_images/2.png', cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread('origin_images/222.png', cv2.IMREAD_GRAYSCALE)

In [5]:

# 1. 特征提取
keypoints1, descriptors1 = extract_features(image1)
keypoints2, descriptors2 = extract_features(image2)

# 2. 特征匹配
good_matches = match_features_bf(descriptors1, descriptors2)

# 3. 变换估计
M = estimate_transform(keypoints1, keypoints2, good_matches)

# 4. 变换应用
transformed_image = apply_transform(image1, M)
cv2.imwrite('trans_image/transformed2to222.png', transformed_image)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\xfeatures2d\src\surf.cpp:1028: error: (-213:The function/feature is not implemented) This algorithm is patented and is excluded in this configuration; Set OPENCV_ENABLE_NONFREE CMake option and rebuild the library in function 'cv::xfeatures2d::SURF::create'
